File name: main.ipynb

Author: ADAMsLEG

Date created: 03/11/2022

Date last modified: 03/11/2022

Python Version: 3.9


In [1]:
import pandas as pd

In [11]:
DATA_PATH = "data/"

INTERVENTIONS_DATASET = DATA_PATH + "interventions.csv"
GLOBAL_MOBILITY_DATASET = DATA_PATH + "Global_Mobility_Report.csv.gz"
TOPICS_LINKED_DATASET = DATA_PATH + "topics_linked.csv.xz"
AGGREGATED_JSON = DATA_PATH + "aggregated_timeseries.json.gz"


In [12]:
df_interventions = pd.read_csv(INTERVENTIONS_DATASET)